Example notebook showing how to use the nested sampler

In [1]:
import os
import sys
import torch
from getdist import plots, MCSamples
import getdist
import numpy as np

In [2]:
path = os.path.realpath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, path)

In [3]:
from nnest import NestedSampler
from nnest.likelihoods import *

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [5]:
# Likelihood + prior
#like = Himmelblau(2)
#transform = lambda x: 5*x
like = Rosenbrock(10)
transform = lambda x: 5*x
#like = Gaussian(2, 0.9)
#transform = lambda x: 3*x
#like = Eggbox(2)
#transform = lambda x: 5*np.pi*x
#like = GaussianShell(2)
#transform = lambda x: 5*x
#like = GaussianMix(2)
#transform = lambda x: 5*x

In [6]:
sampler = NestedSampler(like.x_dim, like, transform=transform, num_live_points=1000, hidden_dim=16, num_blocks=3, flow='spline')

Creating directory for new run logs/test/run18
[nnest.trainer] [INFO] SingleSpeedSpline(
  (flow): NormalizingFlow(
    (flows): ModuleList(
      (0): ActNorm()
      (1): Invertible1x1Conv()
      (2): NSF_CL(
        (f1): MLP(
          (net): Sequential(
            (0): Linear(in_features=5, out_features=16, bias=True)
            (1): LeakyReLU(negative_slope=0.2)
            (2): Linear(in_features=16, out_features=16, bias=True)
            (3): LeakyReLU(negative_slope=0.2)
            (4): Linear(in_features=16, out_features=16, bias=True)
            (5): LeakyReLU(negative_slope=0.2)
            (6): Linear(in_features=16, out_features=115, bias=True)
          )
        )
        (f2): MLP(
          (net): Sequential(
            (0): Linear(in_features=5, out_features=16, bias=True)
            (1): LeakyReLU(negative_slope=0.2)
            (2): Linear(in_features=16, out_features=16, bias=True)
            (3): LeakyReLU(negative_slope=0.2)
            (4): Linear(in_f

In [ ]:
sampler.run(strategy=['rejection_prior', 'mcmc'])

[nnest.sampler] [INFO] MCMC steps [50]
[nnest.sampler] [INFO] Initial scale [0.6325]
[nnest.sampler] [INFO] Volume switch [-1.0000]
[nnest.sampler] [INFO] Step [0] loglstar [-3.8737e+05] max logl [-9.0688e+03] logz [-3.8737e+05] vol [1.00000e+00] ncalls [1001] mean calls [0.0000]
[nnest.sampler] [INFO] Step [200] loglstar [-1.7131e+05] max logl [-9.0688e+03] logz [-1.7132e+05] vol [8.18731e-01] ncalls [1216] mean calls [1.2000]
[nnest.sampler] [INFO] Step [400] loglstar [-1.4193e+05] max logl [-9.0688e+03] logz [-1.4194e+05] vol [6.70320e-01] ncalls [1494] mean calls [1.3000]
[nnest.sampler] [INFO] Step [600] loglstar [-1.2401e+05] max logl [-9.0688e+03] logz [-1.2402e+05] vol [5.48812e-01] ncalls [1819] mean calls [1.5500]
[nnest.sampler] [INFO] Step [800] loglstar [-1.0887e+05] max logl [-9.0688e+03] logz [-1.0888e+05] vol [4.49329e-01] ncalls [2205] mean calls [2.1000]
[nnest.sampler] [INFO] Step [1000] loglstar [-9.7810e+04] max logl [-9.0688e+03] logz [-9.7818e+04] vol [3.67879e-0

[nnest.sampler] [INFO] Acceptance [0.5420] min ESS [3.8995] max ESS [11.9810] average jump [0.1668]
[nnest.sampler] [INFO] Step [6800] loglstar [-7.9392e+03] maxlogl [-7.0298e+02] logz [-7.9529e+03] vol [1.11378e-03] ncalls [156937] scale [0.4263]
[nnest.trainer] [INFO] Number of training samples [1000]
[nnest.trainer] [INFO] Training jitter [0.0482]
[nnest.trainer] [INFO] Epoch [1] train loss [0.0110] validation loss [0.0097]
[nnest.trainer] [INFO] Epoch [56] ran out of patience
[nnest.trainer] [INFO] Best epoch [6] validation loss [0.0086] train time (s) [18.2133]]
[nnest.sampler] [INFO] Acceptance [0.5320] min ESS [2.2472] max ESS [7.5194] average jump [0.1233]
[nnest.sampler] [INFO] Step [7000] loglstar [-7.3770e+03] maxlogl [-7.0298e+02] logz [-7.3909e+03] vol [9.11882e-04] ncalls [164202] scale [0.1760]
[nnest.sampler] [INFO] Acceptance [0.5180] min ESS [1.5459] max ESS [11.7539] average jump [0.1173]
[nnest.sampler] [INFO] Step [7200] loglstar [-6.7762e+03] maxlogl [-7.0034e+02]

[nnest.trainer] [INFO] Training jitter [0.0298]
[nnest.trainer] [INFO] Epoch [1] train loss [-0.0371] validation loss [-0.0415]
[nnest.trainer] [INFO] Epoch [65] ran out of patience
[nnest.trainer] [INFO] Best epoch [15] validation loss [-0.0417] train time (s) [23.8779]]
[nnest.sampler] [INFO] Acceptance [0.5060] min ESS [4.1942] max ESS [17.3517] average jump [0.0964]
[nnest.sampler] [INFO] Step [12000] loglstar [-1.2035e+03] maxlogl [-7.1781e+01] logz [-1.2213e+03] vol [6.14421e-06] ncalls [347480] scale [0.3107]
[nnest.sampler] [INFO] Acceptance [0.5180] min ESS [4.5348] max ESS [9.3312] average jump [0.0713]
[nnest.sampler] [INFO] Step [12200] loglstar [-1.1290e+03] maxlogl [-7.1781e+01] logz [-1.1470e+03] vol [5.03046e-06] ncalls [354983] scale [0.4795]
[nnest.sampler] [INFO] Acceptance [0.5100] min ESS [3.3530] max ESS [9.8057] average jump [0.0812]
[nnest.sampler] [INFO] Step [12400] loglstar [-1.0549e+03] maxlogl [-7.1781e+01] logz [-1.0736e+03] vol [4.11859e-06] ncalls [36228

[nnest.sampler] [INFO] Step [17000] loglstar [-2.8367e+02] maxlogl [-5.0739e+01] logz [-3.0483e+02] vol [4.13994e-08] ncalls [534289] scale [0.2090]
[nnest.sampler] [INFO] Acceptance [0.6160] min ESS [2.1051] max ESS [17.5287] average jump [0.0474]
[nnest.sampler] [INFO] Step [17200] loglstar [-2.7044e+02] maxlogl [-5.0739e+01] logz [-2.9160e+02] vol [3.38949e-08] ncalls [541835] scale [0.4320]
[nnest.sampler] [INFO] Acceptance [0.4820] min ESS [10.3245] max ESS [38.9810] average jump [0.0574]
[nnest.sampler] [INFO] Step [17400] loglstar [-2.5847e+02] maxlogl [-5.0739e+01] logz [-2.7981e+02] vol [2.77508e-08] ncalls [549355] scale [0.2452]
[nnest.sampler] [INFO] Acceptance [0.5640] min ESS [14.7832] max ESS [33.5112] average jump [0.0511]
[nnest.sampler] [INFO] Step [17600] loglstar [-2.4718e+02] maxlogl [-3.4552e+01] logz [-2.6863e+02] vol [2.27205e-08] ncalls [556693] scale [0.1924]
[nnest.sampler] [INFO] Acceptance [0.4380] min ESS [10.1526] max ESS [42.2420] average jump [0.0527]
[

[nnest.sampler] [INFO] Step [22200] loglstar [-8.7922e+01] maxlogl [-2.7819e+01] logz [-1.1291e+02] vol [2.28382e-10] ncalls [732074] scale [0.3521]
[nnest.sampler] [INFO] Acceptance [0.5560] min ESS [7.7479] max ESS [51.0000] average jump [0.0327]
[nnest.sampler] [INFO] Step [22400] loglstar [-8.4869e+01] maxlogl [-2.7819e+01] logz [-1.0992e+02] vol [1.86984e-10] ncalls [739740] scale [0.2625]
[nnest.sampler] [INFO] Acceptance [0.5460] min ESS [13.4291] max ESS [51.0000] average jump [0.0336]
[nnest.sampler] [INFO] Step [22600] loglstar [-8.1427e+01] maxlogl [-2.7819e+01] logz [-1.0668e+02] vol [1.53089e-10] ncalls [747437] scale [0.3285]
[nnest.sampler] [INFO] Acceptance [0.5860] min ESS [9.2360] max ESS [51.0000] average jump [0.0272]
[nnest.sampler] [INFO] Step [22800] loglstar [-7.8393e+01] maxlogl [-2.7819e+01] logz [-1.0379e+02] vol [1.25339e-10] ncalls [755275] scale [0.2351]
[nnest.trainer] [INFO] Number of training samples [1000]
[nnest.trainer] [INFO] Training jitter [0.0102

[nnest.sampler] [INFO] Acceptance [0.5760] min ESS [7.6070] max ESS [51.0000] average jump [0.0209]
[nnest.sampler] [INFO] Step [27600] loglstar [-3.2678e+01] maxlogl [-1.2649e+01] logz [-6.1897e+01] vol [1.03151e-12] ncalls [940238] scale [0.3770]
[nnest.sampler] [INFO] Acceptance [0.5400] min ESS [13.4336] max ESS [51.0000] average jump [0.0226]
[nnest.sampler] [INFO] Step [27800] loglstar [-3.1568e+01] maxlogl [-1.2570e+01] logz [-6.0929e+01] vol [8.44527e-13] ncalls [948247] scale [0.1450]
[nnest.trainer] [INFO] Number of training samples [1000]
[nnest.trainer] [INFO] Training jitter [0.0061]
[nnest.trainer] [INFO] Epoch [1] train loss [-0.1952] validation loss [-0.1985]
[nnest.trainer] [INFO] Epoch [66] ran out of patience
[nnest.trainer] [INFO] Best epoch [16] validation loss [-0.1994] train time (s) [20.7312]]
[nnest.sampler] [INFO] Acceptance [0.5020] min ESS [8.2556] max ESS [51.0000] average jump [0.0215]
[nnest.sampler] [INFO] Step [28000] loglstar [-3.0520e+01] maxlogl [-1.

[nnest.sampler] [INFO] Step [32800] loglstar [-1.5279e+01] maxlogl [-5.0039e+00] logz [-4.8237e+01] vol [5.69038e-15] ncalls [1145886] scale [0.2120]
[nnest.trainer] [INFO] Number of training samples [1000]
[nnest.trainer] [INFO] Training jitter [0.0038]
[nnest.trainer] [INFO] Epoch [1] train loss [-0.2420] validation loss [-0.2459]
[nnest.trainer] [INFO] Epoch [98] ran out of patience
[nnest.trainer] [INFO] Best epoch [48] validation loss [-0.2466] train time (s) [33.9361]]
[nnest.sampler] [INFO] Acceptance [0.5560] min ESS [8.8139] max ESS [51.0000] average jump [0.0237]
[nnest.sampler] [INFO] Step [33000] loglstar [-1.4902e+01] maxlogl [-4.4904e+00] logz [-4.8008e+01] vol [4.65889e-15] ncalls [1153854] scale [0.3695]
[nnest.sampler] [INFO] Acceptance [0.5500] min ESS [11.0376] max ESS [51.0000] average jump [0.0223]
[nnest.sampler] [INFO] Step [33200] loglstar [-1.4543e+01] maxlogl [-4.4904e+00] logz [-4.7794e+01] vol [3.81437e-15] ncalls [1161682] scale [0.1541]
[nnest.sampler] [IN

[nnest.trainer] [INFO] Number of training samples [1000]
[nnest.trainer] [INFO] Training jitter [0.0024]
[nnest.trainer] [INFO] Epoch [1] train loss [-0.2919] validation loss [-0.2978]
[nnest.trainer] [INFO] Epoch [84] ran out of patience
[nnest.trainer] [INFO] Best epoch [34] validation loss [-0.2990] train time (s) [32.2350]]
[nnest.sampler] [INFO] Acceptance [0.5460] min ESS [2.8388] max ESS [51.0000] average jump [0.0220]
[nnest.sampler] [INFO] Step [38000] loglstar [-7.5367e+00] maxlogl [-1.7735e+00] logz [-4.4581e+01] vol [3.13913e-17] ncalls [1353436] scale [0.4301]


In [ ]:
print(sampler.logz)

In [ ]:
mc = MCSamples(samples=sampler.samples, weights=sampler.weights, loglikes=-sampler.loglikes)

In [ ]:
print(mc.getEffectiveSamples())
print(mc.getMargeStats())
print(mc.likeStats)

In [ ]:
g = plots.getSubplotPlotter(width_inch=8)
g.triangle_plot(mc, filled=True)